In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import pickle
import seaborn as sns
import time
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats
import scipy.optimize

In [2]:
# simulation functions
def update(state, fitness, next_gen, burst_size, mut_prob):
    num_replicating = math.ceil(next_gen/burst_size)
    rep_probs = np.multiply(state, fitness)
    rep_probs = rep_probs/np.sum(rep_probs)
    rep_viruses = np.random.choice(len(fitness), size=num_replicating, replace=True, p=rep_probs)
    unique, counts = np.unique(rep_viruses, return_counts=True)

    to_select, select_probs = np.unique(rep_viruses, return_counts=True)
    survivors = np.random.choice(to_select, size=int(next_gen), replace=True, p=select_probs/np.sum(select_probs))
    
    unique, counts = np.unique(survivors, return_counts=True)
    mutators = np.random.choice(survivors, size=np.random.binomial(next_gen, mut_prob), replace=False)
    
    unique, counts = np.unique(mutators, return_counts=True)
    to_return = []
    for i in range(len(fitness)):
        new_pop = np.sum(survivors == i)
        new_pop += np.sum(mutators == (i-1))
        new_pop -= np.sum(mutators == i)
        to_return.append(new_pop)
    return(np.array(to_return))

def simulate(viral_load_curve, fitness_cost, fitness_benefit, num_mut, mut_prob, burst_size=1e3):
    fitnesses = np.zeros(num_mut+1) + 1
    if num_mut > 1:
        fitnesses[1:(num_mut-1)] -= fitness_cost
    fitnesses[-1] += fitness_benefit
    curr_state = np.zeros(num_mut+1)
    curr_state[0] = viral_load_curve[0]
    all_data = np.zeros((len(viral_load_curve), num_mut+1))
    all_data[0,:] = curr_state
    for i in range(1,len(viral_load_curve)):
        load = viral_load_curve[i]
        curr_state = update(curr_state, fitnesses, load, burst_size, mut_prob)
        all_data[i,:] = curr_state
    return(all_data)

In [3]:
# functions for processing and plotting simulation data
def compute_fracs(data, log_data=True, CI=None, median=False):
    all_data = np.array(data)
    
    kinetics = np.sum(data[0], axis=1)
    kinetics = np.reshape(kinetics, (len(kinetics),1))
    all_data = all_data/kinetics
    if log_data:
        all_data = np.log10(all_data)
        all_data[all_data == -np.inf] = -7
    
    if median:
        means = np.quantile(all_data, 0.5, axis=0)
    else:
        means = np.nanmean(all_data, axis=0)
    if CI is None:
        sem = stats.sem(means, nan_policy="omit")
        lowers = means - sem
        uppers = means + sem
    else:
        lowers = np.quantile(all_data, (1-CI)/2, axis=0)
        uppers = np.quantile(all_data, 1-(1-CI)/2, axis=0)
    return([means, lowers, uppers])

def transmit_probs(data, end_time=None, num_trans=100):
    all_data = np.array(data)
    
    if not end_time is None:
        all_data = all_data[:,0:int(end_time*2),:]
    total_load = np.sum(all_data[0,:,:])
    
    pt_probs = np.sum(all_data, axis=1)/total_load
    pt_probs = 1-np.power((1-pt_probs), num_trans)
    
    return(pt_probs)

def max_freq(data):
    all_data = np.array(data)
    
    kinetics = np.sum(data[0], axis=1)
    kinetics = np.reshape(kinetics, (len(kinetics),1))
    all_data = all_data/kinetics

    maxes = np.quantile(all_data, 1, axis=1)
    return(np.mean(maxes, axis=0))

In [ ]:
# location to save intermediate results
save_dir = 

In [ ]:
# building viral load curves for vaccinated and unvaccinated individuals
# using data from Table S5 in https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(21)00648-4/fulltext

#peak_VL = 10**8.14
peak_VL = 10**9.14
tmax = 3 #days; taken from main text
growth_unvax = 6.46
growth_vax = 6.2
decline_unvax = 1.82
decline_vax = 2.19

def calc_VL(time, growth, decline):
    to_return = peak_VL * (growth+decline)
    to_return = to_return/(decline*np.exp(-growth*(time-tmax)) + growth*np.exp(decline*(time-tmax)))
    return(to_return)

tvals = np.arange(0,14, step=0.5)
vax_VL = calc_VL(tvals, growth_vax, decline_vax)
unvax_VL = calc_VL(tvals, growth_unvax, decline_unvax)

In [ ]:
# Fig. 5A: plotting
plt.figure()
plt.plot(tvals, vax_VL, color="blueviolet")
plt.plot(tvals, unvax_VL, color="green")
plt.yscale("log")
plt.ylim(1,peak_VL*2)
plt.ylabel("Total viral load", fontsize=14)
plt.xlabel("Days after infection", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
sns.despine()
plt.tight_layout()
#plt.savefig(..., transparent=True)
plt.show()

In [ ]:
# Simulate and save data for Fig. 5B-C
viral_load_kinetics = vax_VL
fitness_benefits = [-0.01, -0.05, 0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
viral_load_adj = np.array(viral_load_kinetics)/1000
viral_load_adj = viral_load_adj[viral_load_adj>1]

num_iter = 1000

for benefit in fitness_benefits:
    print(benefit)
    all_res = []
    for i in range(num_iter):
        all_res.append(simulate(viral_load_adj, None, benefit, 1, 1e-5, burst_size=1))
    pickle.dump(all_res, open(save_dir+"vax_"+str(benefit)+".p", "wb" ))
    

viral_load_kinetics = unvax_VL
viral_load_adj = np.array(viral_load_kinetics)/1000
viral_load_adj = viral_load_adj[viral_load_adj>1]
for benefit in fitness_benefits:
    print(benefit)
    all_res = []
    for i in range(num_iter):
        all_res.append(simulate(viral_load_adj, None, benefit, 1, 1e-5, burst_size=1))
    pickle.dump(all_res, open(save_dir+"unvax_"+str(benefit)+".p", "wb" ))

In [ ]:
# Load data for Fig. 5B-C
fitness_benefits = [-0.05, -0.01, 0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

means = []
lowers = []
uppers = []
early_probs = []
mid_probs = []
midlate_probs = []
late_probs = []
for benefit in fitness_benefits:
    data = pickle.load(open(save_dir+"unvax_"+str(benefit)+".p", "rb" ))
    mean, lower, upper = compute_fracs(data, log_data=False, CI=None, median=False)
    means.append(mean)
    lowers.append(lower)
    uppers.append(upper)
    early_probs.append(transmit_probs(data, 3, num_trans=10))
    mid_probs.append(transmit_probs(data, 5, num_trans=10))
    midlate_probs.append(transmit_probs(data, 7, num_trans=10))
    late_probs.append(transmit_probs(data, 23, num_trans=10))
    
unvax_means = means
unvax_lowers = lowers
unvax_uppers = uppers
unvax_prob = pd.DataFrame({"benefit":fitness_benefits, "day3":[np.mean(x[:,1]) for x in early_probs], "day5":[np.mean(x[:,1]) for x in mid_probs], "day7":[np.mean(x[:,1]) for x in midlate_probs], 'any':[np.mean(x[:,1]) for x in late_probs]})
unvax_prob_SEMs = pd.DataFrame({"benefit":fitness_benefits, "day3":[stats.sem(x[:,1]) for x in early_probs], "day5":[stats.sem(x[:,1]) for x in mid_probs], "day7":[stats.sem(x[:,1]) for x in midlate_probs], 'any':[stats.sem(x[:,1]) for x in late_probs]})
unvax_prob.sort_values(by="benefit", inplace=True)
unvax_prob_SEMs.sort_values(by="benefit", inplace=True)

means = []
lowers = []
uppers = []
early_probs = []
mid_probs = []
midlate_probs = []
late_probs = []
for benefit in fitness_benefits:
    data = pickle.load(open(save_dir+"vax_"+str(benefit)+".p", "rb" ))
    mean, lower, upper = compute_fracs(data, log_data=False, CI=None, median=False)
    means.append(mean)
    lowers.append(lower)
    uppers.append(upper)
    early_probs.append(transmit_probs(data, 3, num_trans=10))
    mid_probs.append(transmit_probs(data, 5, num_trans=10))
    midlate_probs.append(transmit_probs(data, 7, num_trans=10))
    late_probs.append(transmit_probs(data, 23, num_trans=10))
    
vax_means = means
vax_lowers = lowers
vax_uppers = uppers
vax_prob = pd.DataFrame({"benefit":fitness_benefits, "day3":[np.mean(x[:,1]) for x in early_probs], "day5":[np.mean(x[:,1]) for x in mid_probs], "day7":[np.mean(x[:,1]) for x in midlate_probs], 'any':[np.mean(x[:,1]) for x in late_probs]})
vax_prob_SEMs = pd.DataFrame({"benefit":fitness_benefits, "day3":[stats.sem(x[:,1]) for x in early_probs], "day5":[stats.sem(x[:,1]) for x in mid_probs], "day7":[stats.sem(x[:,1]) for x in midlate_probs], 'any':[stats.sem(x[:,1]) for x in late_probs]})
vax_prob.sort_values(by="benefit", inplace=True)
vax_prob_SEMs.sort_values(by="benefit", inplace=True)

In [ ]:
# plotting Fig. 5B

benefit_index = 6
mut_index = 1

plt.figure()
xplot = np.arange(np.shape(vax_means[benefit_index][:,mut_index])[0])/2
plt.plot(xplot, vax_means[benefit_index][:,mut_index], color="blueviolet")
plt.fill_between(xplot, vax_lowers[benefit_index][:,mut_index], vax_uppers[benefit_index][:,mut_index], color="blueviolet", alpha=0.3)

#xplot = np.arange(np.shape(unvax_means[benefit_index][:,mut_index])[0])/2
plt.plot(xplot, unvax_means[benefit_index][:,mut_index][0:len(xplot)], color="green")
plt.fill_between(xplot, unvax_lowers[benefit_index][:,mut_index][0:len(xplot)], unvax_uppers[benefit_index][:,mut_index][0:len(xplot)], color="green", alpha=0.3)

#plt.yscale("log")
plt.ylabel("Mean intrahost\nfrequency of variant", fontsize=14)
plt.xlabel("Days since infection", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(0,0.0014)
sns.despine()
plt.tight_layout()
#plt.savefig(... transparent=True)
plt.show()

In [ ]:
# plotting Fig. 5C
plt.figure()
plt.plot(vax_prob["benefit"], vax_prob["any"], color="blueviolet")
plt.fill_between(vax_prob["benefit"], vax_prob["any"]-vax_prob_SEMs["any"], vax_prob["any"]+vax_prob_SEMs["any"], color="blueviolet", alpha=0.3)
plt.plot(unvax_prob["benefit"], unvax_prob["any"], color="green")
plt.fill_between(unvax_prob["benefit"], unvax_prob["any"]-unvax_prob_SEMs["any"], unvax_prob["any"]+unvax_prob_SEMs["any"], color="green", alpha=0.3)
plt.ylabel("Probability of passing on variant", fontsize=14)
plt.xlabel("Fitness effect of mutation", fontsize=14)
#plt.yscale("log")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(0,0.0014)
sns.despine()
plt.tight_layout()
#plt.savefig(..., transparent=True)
plt.show()

In [ ]:
# Simulate and save data for Fig. 5D
viral_load_kinetics = vax_VL
fitness_benefits = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
valley = 0.01
viral_load_adj = np.array(viral_load_kinetics)/1000
viral_load_adj = viral_load_adj[viral_load_adj>1]

num_iter = 1000

for benefit in fitness_benefits:
    print(benefit)
    all_res = []
    for i in range(num_iter):
        all_res.append(simulate(viral_load_adj, valley, benefit, 2, 1e-5, burst_size=1))
    pickle.dump(all_res, open(save_dir+"valley_vax_"+str(benefit)+".p", "wb" ))
    

viral_load_kinetics = unvax_VL
viral_load_adj = np.array(viral_load_kinetics)/1000
viral_load_adj = viral_load_adj[viral_load_adj>1]
for benefit in fitness_benefits:
    print(benefit)
    all_res = []
    for i in range(num_iter):
        all_res.append(simulate(viral_load_adj, valley, benefit, 2, 1e-5, burst_size=1))
    pickle.dump(all_res, open(save_dir+"valley_unvax_"+str(benefit)+".p", "wb" ))

In [ ]:
# Load data for Fig. 5D

fitness_benefits = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

means = []
lowers = []
uppers = []
early_probs = []
mid_probs = []
midlate_probs = []
late_probs = []
for benefit in fitness_benefits:
    data = pickle.load(open(save_dir+"valley_unvax_"+str(benefit)+".p", "rb" ))
    mean, lower, upper = compute_fracs(data, log_data=False, CI=None, median=False)
    means.append(mean)
    lowers.append(lower)
    uppers.append(upper)
    early_probs.append(transmit_probs(data, 3, num_trans=10))
    mid_probs.append(transmit_probs(data, 5, num_trans=10))
    midlate_probs.append(transmit_probs(data, 7, num_trans=10))
    late_probs.append(transmit_probs(data, 23, num_trans=10))
    
unvax_means = means
unvax_lowers = lowers
unvax_uppers = uppers
unvax_prob = pd.DataFrame({"benefit":fitness_benefits, "day3":[np.mean(x[:,1]) for x in early_probs], "day5":[np.mean(x[:,1]) for x in mid_probs], "day7":[np.mean(x[:,1]) for x in midlate_probs], 'any':[np.mean(x[:,1]) for x in late_probs]})
unvax_prob_SEMs = pd.DataFrame({"benefit":fitness_benefits, "day3":[stats.sem(x[:,1]) for x in early_probs], "day5":[stats.sem(x[:,1]) for x in mid_probs], "day7":[stats.sem(x[:,1]) for x in midlate_probs], 'any':[stats.sem(x[:,1]) for x in late_probs]})
unvax_prob.sort_values(by="benefit", inplace=True)
unvax_prob_SEMs.sort_values(by="benefit", inplace=True)

means = []
lowers = []
uppers = []
early_probs = []
mid_probs = []
midlate_probs = []
late_probs = []
for benefit in fitness_benefits:
    data = pickle.load(open(save_dir+"valley_vax_"+str(benefit)+".p", "rb" ))
    mean, lower, upper = compute_fracs(data, log_data=False, CI=None, median=False)
    means.append(mean)
    lowers.append(lower)
    uppers.append(upper)
    early_probs.append(transmit_probs(data, 3, num_trans=10))
    mid_probs.append(transmit_probs(data, 5, num_trans=10))
    midlate_probs.append(transmit_probs(data, 7, num_trans=10))
    late_probs.append(transmit_probs(data, 23, num_trans=10))
    
vax_means = means
vax_lowers = lowers
vax_uppers = uppers
vax_prob = pd.DataFrame({"benefit":fitness_benefits, "day3":[np.mean(x[:,1]) for x in early_probs], "day5":[np.mean(x[:,1]) for x in mid_probs], "day7":[np.mean(x[:,1]) for x in midlate_probs], 'any':[np.mean(x[:,1]) for x in late_probs]})
vax_prob_SEMs = pd.DataFrame({"benefit":fitness_benefits, "day3":[stats.sem(x[:,1]) for x in early_probs], "day5":[stats.sem(x[:,1]) for x in mid_probs], "day7":[stats.sem(x[:,1]) for x in midlate_probs], 'any':[stats.sem(x[:,1]) for x in late_probs]})
vax_prob.sort_values(by="benefit", inplace=True)
vax_prob_SEMs.sort_values(by="benefit", inplace=True)

In [ ]:
# plotting Fig. 5D
plt.figure()
plt.plot(vax_prob["benefit"], vax_prob["any"], color="blueviolet")
plt.fill_between(vax_prob["benefit"], vax_prob["any"]-vax_prob_SEMs["any"], vax_prob["any"]+vax_prob_SEMs["any"], color="blueviolet", alpha=0.3)
plt.plot(unvax_prob["benefit"], unvax_prob["any"], color="green")
plt.fill_between(unvax_prob["benefit"], unvax_prob["any"]-unvax_prob_SEMs["any"], unvax_prob["any"]+unvax_prob_SEMs["any"], color="green", alpha=0.3)
plt.ylabel("Probability of passing on variant", fontsize=14)
plt.xlabel("Fitness effect of mutation", fontsize=14)
#plt.yscale("log")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.ylim(0.0003,0.0007)
sns.despine()
plt.tight_layout()
#plt.savefig(..., transparent=True)
plt.show()

In [ ]:
# Simulate and save data for Fig. 6

extra_weeks = [0, 1, 2, 3, 4, 5, 10, 15]
valley = 0.01
benefit = 0.5

num_iter = 1000

for time in extra_weeks:
    print(time)
    all_res = []
    viral_load_kinetics = vax_VL
    viral_load_adj = np.array(viral_load_kinetics)/1000
    viral_load_adj = viral_load_adj[viral_load_adj>1]
    left_kinetics = viral_load_adj[0:6]
    right_kinetics = viral_load_adj[7:]
    new_viral_load = np.concatenate([left_kinetics, [np.max(viral_load_adj)]*(time*14), right_kinetics], axis=None)
    for i in range(num_iter):
        all_res.append(simulate(new_viral_load, valley, benefit, 2, 1e-5, burst_size=1))
    pickle.dump(all_res, open(save_dir+"valley_vax_long_0.5_"+str(time)+".p", "wb" ))

    all_res = []
    viral_load_kinetics = unvax_VL
    viral_load_adj = np.array(viral_load_kinetics)/1000
    viral_load_adj = viral_load_adj[viral_load_adj>1]
    left_kinetics = viral_load_adj[0:6]
    right_kinetics = viral_load_adj[7:]
    new_viral_load = np.concatenate([left_kinetics, [np.max(viral_load_adj)]*(time*14), right_kinetics], axis=None)
    for i in range(num_iter):
        all_res.append(simulate(new_viral_load, valley, benefit, 2, 1e-5, burst_size=1))
    pickle.dump(all_res, open(save_dir+"valley_unvax_long_0.5_"+str(time)+".p", "wb" ))

In [ ]:
# Load data for Fig. 6
extra_weeks = [0, 1, 2, 3, 4, 5, 10, 15]

means = []
lowers = []
uppers = []
trans_probs = []
for week in extra_weeks:
    data = pickle.load(open(save_dir+"valley_vax_long_0.5_"+str(week)+".p", "rb" ))
    mean, lower, upper = compute_fracs(data, log_data=False, CI=None, median=False)
    means.append(mean)
    lowers.append(lower)
    uppers.append(upper)
    trans_probs.append(transmit_probs(data, num_trans=10))
    
to_plot_hold = pd.DataFrame({"week":extra_weeks, 'any':[np.mean(x[:,2]) for x in trans_probs]})
SEMS_hold = pd.DataFrame({"week":extra_weeks, 'any':[stats.sem(x[:,2]) for x in trans_probs]})
to_plot_hold.sort_values(by="week", inplace=True)
SEMS_hold.sort_values(by="week", inplace=True)
uppers_hold = uppers
lowers_hold = lowers
means_hold = means

means = []
lowers = []
uppers = []
trans_probs = []
for week in extra_weeks:
    data = pickle.load(open(save_dir+"valley_unvax_long_0.5_"+str(week)+".p", "rb" ))
    mean, lower, upper = compute_fracs(data, log_data=False, CI=None, median=False)
    means.append(mean)
    lowers.append(lower)
    uppers.append(upper)
    trans_probs.append(transmit_probs(data, num_trans=10))
    
to_plot = pd.DataFrame({"week":extra_weeks, 'any':[np.mean(x[:,2]) for x in trans_probs]})
prob_SEMs = pd.DataFrame({"week":extra_weeks, 'any':[stats.sem(x[:,2]) for x in trans_probs]})
to_plot.sort_values(by="week", inplace=True)
prob_SEMs.sort_values(by="week", inplace=True)

In [ ]:
# plotting Fig. 6A
plt.figure()
mut_index = 2
i = 7
xplot = np.arange(len(means[i][:,mut_index]))/(2*7)
plt.plot(xplot, means[i][:,mut_index], color="green")
plt.fill_between(xplot, lowers[i][:,mut_index], uppers[i][:,mut_index], color="green", alpha=0.3)

mut_index = 2
i = 7
xplot = np.arange(len(means_hold[i][:,mut_index]))/(2*7)
plt.plot(xplot, means_hold[i][:,mut_index], color="blueviolet")
plt.fill_between(xplot, lowers_hold[i][:,mut_index], uppers_hold[i][:,mut_index], color="blueviolet", alpha=0.3)
    
#plt.yscale("log")
plt.ylabel("Mean intrahost\nfrequency of variant", fontsize=14)
plt.xlabel("Weeks since infection started", fontsize=14)
plt.xticks(ticks = [0,2,4,6,8,10,12,14,16,18], fontsize=14)
plt.yticks(fontsize=14)

sns.despine()
plt.tight_layout()
#plt.savefig(..., transparent=True)
plt.show()

In [ ]:
# plotting Fig. 6B
plt.figure()
plt.plot(to_plot_hold["week"]+3, to_plot_hold["any"], color="blueviolet")
plt.fill_between(to_plot_hold["week"]+3, to_plot_hold["any"]-SEMS_hold["any"], to_plot_hold["any"]+SEMS_hold["any"], color="blueviolet", alpha=0.3)

plt.plot(to_plot["week"]+3, to_plot["any"], color="green")
plt.fill_between(to_plot["week"]+3, to_plot["any"]-prob_SEMs["any"], to_plot["any"]+prob_SEMs["any"], color="green", alpha=0.3)
plt.ylabel("Probability of passing on variant", fontsize=14)
plt.xlabel("Infection length (wks)", fontsize=14)
plt.yscale("log")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.ylim(1e-10, 2e-1)
sns.despine()
plt.tight_layout()
#plt.savefig(..., transparent=True)
plt.show()

In [ ]:
# plotting Fig. 6C

# data for infection length distribution from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7734137/

long_lower_IQR = 86
long_upper_IQR = 101.5
est_SD = (long_upper_IQR-long_lower_IQR)/1.35
est_mean = (long_upper_IQR+long_lower_IQR)/2

from scipy.stats import norm
total_lens = np.array(extra_weeks) * 7 + 23-5
cdf_vals = norm.cdf(total_lens, loc=est_mean, scale=est_SD)
long_dist = cdf_vals[1:] - cdf_vals[:-1]


lam = 1.5
def find_pext(x):
    return(1-np.exp(-lam*x) - x)
N = 5e4
p_surv = scipy.optimize.broyden1(find_pext, 0.1, f_tol=1e-14)
long_nvar_2mut = np.sum(to_plot_hold["any"][1:] * long_dist * p_surv*N)
short_nvar_2mut = to_plot_hold["any"][0]

frac_long = [0.0001, 0.001, 0.01, 0.05]
combined_nvar_vax = [short_nvar_2mut*(1-p) + long_nvar_2mut*p for p in frac_long]
label_percent = [str(x)+"%" for x in np.array(frac_long)*100]

long_nvar_2mut = np.sum(to_plot["any"][1:] * long_dist * p_surv*N)
short_nvar_2mut = to_plot["any"][0]
combined_nvar_unvax = [short_nvar_2mut*(1-p) + long_nvar_2mut*p for p in frac_long]

dodge = 0.15
width = 0.3
plt.figure()
plt.bar(x=np.arange(len(frac_long))-dodge, height=combined_nvar, color="blueviolet", width=width)
plt.bar(x=np.arange(len(frac_long))+dodge, height=combined_nvar_unvax, color="green", width=width)

plt.xticks(ticks=range(len(frac_long)), labels=label_percent, fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Number of infections/day with\nnew two-mutation combination", fontsize=14)
plt.xlabel("Long-term viral shedder frequency in population", fontsize=14)
sns.despine()
plt.tight_layout()
#plt.savefig(..., transparent=True)
plt.show()